## Prototype for `napari` in a web-browser using `fastplotlib` as the front-end

In [1]:
import fastplotlib as fpl
import napari
import numpy as np
import imageio.v3 as iio
from ipywidgets import IntSlider, VBox
from functools import partial

### Create a 2D image in `napari` and view in `fastplotlib`

In [2]:
data = iio.imread('imageio:camera.png')

In [3]:
napari_img = napari.layers.Image(data=data)

In [4]:
napari_img.contrast_limits

[0, 255]

#### Simple `napari` 2D image_wrapper using `fastplotlib`

In [5]:
def image_wrapper(image: napari.layers.Image) -> fpl.graphics.ImageGraphic:
    """Return fastplotlib ImageGraphic based on napari Image layer.
    
    Parameters
    ==========
    image: napari.layers.Image
        napari Image layer to wrap
    
    Returns
    =======
    fpl.graphics.ImageGraphic
    """
    image_graphic = fpl.graphics.ImageGraphic(data=image.data, 
                        name=image.name, 
                        metadata=image.metadata,
                        cmap=image.colormap.name,
                        vmin=image.contrast_limits[0],
                        vmax=image.contrast_limits[1]
                       )
    
    # event handler to update contrast limits
    def update_vmin_vmax(ev):
        image_graphic.cmap.vmin, image_graphic.cmap.vmax = ev.source.contrast_limits
    
    # link napari contrast limits to update fpl plot
    image.events.contrast_limits.connect(update_vmin_vmax)
    
    return image_graphic

In [6]:
# create fastplotlib plot instance
plot = fpl.Plot()

# wrap napari image
fpl_img = image_wrapper(image=napari_img)

# add wrapped image to plot 
plot.add_graphic(fpl_img)

# view plot 
plot.show()

RFBOutputContext()

In [7]:
# flip image
plot.camera.world.scale_y *= -1

#### Update contrast limits of `napari` image

In [8]:
napari_img.contrast_limits = (10, 200)

### Create `fastplotlib` viewer that wraps `napari` viewer

In [9]:
class FplViewer(napari.components.ViewerModel):
    # napari uses pydantic, so must declare subclass specific attributes here
    plot: fpl.Plot = None
    sliders: list = None
    def __init__(
                self,
                *args,
                **kwargs):
        """
        Fastplotlib view wrapper to act as napari front-end viewer. Wraps the 
        napari ViewerModel class. 
        """
        super().__init__(**kwargs)
        
        self.plot = fpl.Plot()
        self.sliders = list()
        
        # connect inserting/removing layers to update fpl viewer
        self._connect_layer_events()
    
    def _add_graphic(self, ev):
        """
        On creation of a new napari layer, will parse the layer type and 
        add corresponding fastplotlib graphic instance. 
        """
        layer_insert_ix = ev.index
        if type(self.layers[layer_insert_ix]).__name__ == "Image":
            print("adding image")
            layer = self.layers[layer_insert_ix]
            # note: data being added is based on the current 2D slice being viewed
            self.plot.add_image(
                data=layer._slice.image.view,
                name=layer.name, 
                metadata=layer.metadata,
                cmap=layer.colormap.name,
                vmin=layer.contrast_limits[0],
                vmax=layer.contrast_limits[1],
            )
        else: # for now only have image implemented
            raise NotImplementedError
    
    def _remove_graphic(self, ev):
        """
        If napari layer removed, will remove corresponding graphic from fpl plot. 
        """
        layer_remove_ix = ev.index
        graphic = self.plot.graphics[layer_remove_ix]
        self.plot.remove_graphic(graphic)
        
    def _update_sliders(self):
        """
        Generate sliders for all dims not being displayed. 
        """
        self.sliders = list()
        
        dims_not_displayed = list(range(self.dims.ndim))[:-2]
        
        for dim in dims_not_displayed[::-1]:
            slider = IntSlider(value=int(self.dims.point[dim]),
                              min=self.dims.range[dim].start,
                              max=self.dims.range[dim].stop,
                              step=self.dims.range[dim].step)
            
            # update data based on dim slider values
            slider.observe(partial(self._update_point, dim), "value")
            self.sliders.append(slider)
        
    def _update_point(self, dim, change):
        """
        Event handler for changes in dim sliders. 
        """
        new_val = change["new"]
        self.dims.set_point(dim, new_val)
        for layer in self.layers:
            self.plot[layer.name].data = layer._slice.image.view
        
    
    def _connect_layer_events(self):
        """
        Connect napari layer events to fpl functions. 
        Must pass position='last' so that napari dims object will get updated
        first so data slice is correct. 
        """
        # insert layer, add graphic and update sliders
        self.layers.events.inserted.connect(self._add_graphic, position="last")
        self.layers.events.inserted.connect(self._update_sliders, position="last")
        # remove layer, remove graphic and update sliders
        self.layers.events.removed.connect(self._remove_graphic)
        self.layers.events.removed.connect(self._update_sliders)
    
    def show(self):
        """
        Return VBox of plot instance and sliders. 
        """
        return VBox([self.plot.show(),
                     *self.sliders]
                   )

In [10]:
data = np.random.rand(12, 16, 18, 22, 24).astype(np.float32)

In [11]:
img = napari.layers.Image(data=data)

In [12]:
fpl_view = FplViewer()

RFBOutputContext()

In [13]:
fpl_view.add_image(data)

adding image


<Image layer 'data' at 0x138f79ff0>

In [14]:
fpl_view.show()

### Check the fastplotlib points quickstart example runs

https://fastplotlib.readthedocs.io/en/latest/quickstart.html#Scatter-plots


In [15]:
# https://fastplotlib.readthedocs.io/en/latest/quickstart.html#Scatter-plots

# create a random distribution
# only 1,000 points shown here in the docs, but it can be millions
n_points = 1_000

# if you have a good GPU go for 1.5 million points :D
# this is multiplied by 3
#n_points = 500_000

# dimensions always have to be [n_points, xyz]
dims = (n_points, 3)

clouds_offset = 15

# create some random clouds
normal = np.random.normal(size=dims, scale=5)
# stack the data into a single array
cloud = np.vstack(
    [
        normal - clouds_offset,
        normal,
        normal + clouds_offset,
    ]
)

# color each of them separately
colors = ["yellow"] * n_points + ["cyan"] * n_points + ["magenta"] * n_points

# create plot
plot_s = fpl.Plot()

# use an alpha value since this will be a lot of points
scatter_graphic = plot_s.add_scatter(data=cloud, sizes=3, colors=colors, alpha=0.7)

plot_s.show()

RFBOutputContext()

/Users/genevieb/mambaforge/envs/napari-demo/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### Create a points layer in `napari` and view in `fastplotlib`

In [16]:
napari_points = napari.layers.Points(cloud, size=3, face_color=colors, opacity=0.5)

#### Simple `napari` points using `fastplotlib`


In [17]:
def points_wrapper(points: napari.layers.Points) -> fpl.graphics.ScatterGraphic:
    """Return fastplotlib ScatterGraphic based on napari Points layer.
    
    Parameters
    ==========
    image: napari.layers.Points
        napari Points layer to wrap
    
    Returns
    =======
    fpl.graphics.ScatterGraphic
    """
    points_graphic = fpl.graphics.ScatterGraphic(data=points.data, 
                        name=points.name, 
                        metadata=points.metadata,
                        sizes=points.size.astype(np.float32),
                        colors=points.face_color.astype(np.float32),
                        alpha=np.array(points.opacity, dtype=np.float32)                                          
                       )
    
    # opacity and point size are not accessible in fastplotlib to connect callbacks :(
    # So we'll connect the less exiting boolean visibility parameter
    def update_visibility(event):
        points_graphic.visible = event.source.visible

    # Link napari visibility to update fpl plot
    points.events.visible.connect(update_visibility)
    
    return points_graphic

In [18]:
# create fastplotlib plot instance
points_plot = fpl.Plot()

# wrap napari image
fpl_points = points_wrapper(points=napari_points)

# add wrapped image to plot 
points_plot.add_graphic(fpl_points)

# view plot 
points_plot.show()

RFBOutputContext()

/Users/genevieb/mambaforge/envs/napari-demo/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


#### Update visibility of `napari` points layer

In [19]:
napari_points.visible = False

In [20]:
napari_points.visible = True

### Extend `fastplotlib` viewer wrapping `napari` viewer to include points

In [21]:
def _add_graphic(self, ev):
    """
    On creation of a new napari layer, will parse the layer type and 
    add corresponding fastplotlib graphic instance. 
    """
    layer_insert_ix = ev.index
    if type(self.layers[layer_insert_ix]).__name__ == "Image":
        print("adding image")
        layer = self.layers[layer_insert_ix]
        # note: data being added is based on the current 2D slice being viewed
        self.plot.add_image(
            data=layer._slice.image.view,
            name=layer.name, 
            metadata=layer.metadata,
            cmap=layer.colormap.name,
            vmin=layer.contrast_limits[0],
            vmax=layer.contrast_limits[1],
        )
    if type(self.layers[layer_insert_ix]).__name__ == "Points":
        print("Adding points")
        layer = self.layers[layer_insert_ix]
        # note: data being added is based on the current 2D slice being viewed
        self.plot.add_scatter(
            data=layer._view_data,
            name=layer.name,
            metadata=layer.metadata,
            sizes=layer._view_size,
            colors=layer._view_face_color,
            alpha=layer.opacity,                                                      
        )
    else: # for now only have image implemented
        raise NotImplementedError


def _update_point(self, dim, change):
    """
    Event handler for changes in dim sliders. 
    """
    new_val = change["new"]
    self.dims.set_point(dim, new_val)
    for layer in self.layers:
        self.plot[layer.name].data = layer._view_data

FplViewer._add_graphic = _add_graphic
FplViewer._update_point = _update_point


In [22]:
n_test_points = 50
test_point_data = None
for sl in range(5):
    random_points = np.random.random(size=(n_test_points,2)) * 10
    z_slice_idx = np.ones((n_test_points, 1)) * sl
    random_points = np.hstack((z_slice_idx, random_points))
    if test_point_data is None:
        test_point_data = random_points
    else:
        test_point_data = np.vstack((test_point_data, random_points))

test_point_data = test_point_data.astype(np.float32)

print(test_point_data.shape)
print(test_point_data[:5])
print("...")
print(test_point_data[-5:])


(250, 3)
[[0.        0.2709095 6.161419 ]
 [0.        4.4294534 1.9165218]
 [0.        4.6046276 1.0556118]
 [0.        9.920743  2.5679748]
 [0.        6.2964187 5.085389 ]]
...
[[4.         7.0594187  6.5985804 ]
 [4.         0.21802871 1.2755091 ]
 [4.         2.1736236  6.0190997 ]
 [4.         6.515993   4.4066334 ]
 [4.         9.196809   6.6120734 ]]


In [23]:
import napari

viewer = napari.Viewer()
viewer.add_points(test_point_data)

<Points layer 'test_point_data' at 0x105a93fd0>

In [40]:
del fpl_view_2

In [41]:
fpl_view_2 = FplViewer()

RFBOutputContext()

In [45]:
fpl_view_2.add_points(test_point_data, size=20)#, face_color=colors, opacity=0.7)

Adding points


/Users/genevieb/mambaforge/envs/napari-demo/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


<Points layer 'test_point_data' at 0x169834820>

In [37]:
rand_2d.shape

(100, 100)

In [44]:
# rand_2d = np.random.random((100,100)) * 10
rand_2d = np.array([[1,1],[2,2],[3,3]], dtype=np.float32)
var = fpl_view_2.plot.add_scatter(rand_2d, sizes=20)

In [43]:
fpl_view_2.show()

In [47]:
var.visible = False

## Notes

* 3d volumes not supported by fastplotlib
* Very few parameters are accessible in fastplotlib to hook callbacks up to (eg: point size, opacity, etc.)
* fastplotlib is much slower and less responsive on my M1 mac than I expected. (I do have `brew install
* Problems when fastplotlib tries to show a scatter plot with no points, or scrolling through a scatter plot with a varying number of points per z slice